In [146]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from gplearn.extra_fitness import _quantile35_longshort_fee
from time import perf_counter

In [148]:
file_data = '/home/pennymax/proj/MFM/CrossSection/v1_basic/results/basic_2023-11-22_22:15:18/fct_man_ta_proc.parquet'
df = pd.read_parquet(file_data)
df

,,AR_26,BCKRET1_ALMA_10_6.0_0.85,BCKRET1_ANGLEd_1,BCKRET1_APO_12_26,BCKRET1_BIAS_SMA_26,BCKRET1_CFO_9,BCKRET1_CG_10,BCKRET1_CMO_14,BCKRET1_COPC_11_14_10,BCKRET1_CTI_12,...,premIdx_close,premIdx_high,premIdx_low,premIdx_open,spot_QuoVol,spot_TakerBuyQuoVol,spotum_quo_rt1,spotum_quo_rt3,spotum_quo_rt5,fwdret1
time,symbol,,,,,,,,,,,,,,,,,,,,,
2022-02-15 00:00:00+00:00,1000BTTCUSDT,0.301435,0.275936,-0.488968,0.771073,-1.161410,0.358964,-0.067306,0.607884,-0.221017,0.085637,...,0.654780,1.305227,-1.578736,0.773644,-0.286571,-0.290574,-1.698317,-1.750555,-1.797949,0.006381
2022-02-15 08:00:00+00:00,1000BTTCUSDT,0.203825,0.346633,-0.102897,0.670855,-0.063863,-0.128800,-0.200930,-0.632266,-0.184119,-0.056086,...,0.654619,1.852459,-2.511996,-3.652448,-0.281280,-0.280721,-1.825539,-1.865662,-1.797600,0.005435
2022-02-15 16:00:00+00:00,1000BTTCUSDT,0.109859,0.194290,-0.281913,-0.625057,-0.035205,-0.048292,-0.183299,-0.913829,-0.256562,-0.479517,...,0.429367,1.520373,-0.490996,0.573695,-0.311542,-0.305104,-1.896369,-1.871790,-1.849618,-0.001351
2022-02-16 00:00:00+00:00,1000BTTCUSDT,-0.026714,0.305321,0.818629,-0.432105,-0.135898,0.066802,-0.001660,0.009961,-0.201409,-0.858671,...,5.207442,2.838670,-1.876175,0.332018,-0.333334,-0.335837,-1.982989,-1.945231,-1.941199,-0.026613
2022-02-16 08:00:00+00:00,1000BTTCUSDT,-0.038897,0.149609,0.062552,-0.885780,-0.009816,-0.113658,0.227472,-0.964727,-0.254134,-0.789266,...,0.377948,2.072545,-2.015252,0.486841,-0.345787,-0.341323,-1.852673,-2.017361,-1.978387,0.007878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-19 08:00:00+00:00,ZRXUSDT,3.664874,-3.113825,-1.648899,-5.691401,-0.116280,-0.002034,0.123369,-1.341036,0.297297,-0.282686,...,-0.606585,-0.537085,-0.761102,-0.556576,-0.180849,-0.182091,2.612331,1.771783,2.564827,0.031750
2023-11-19 16:00:00+00:00,ZRXUSDT,2.537322,-2.549892,1.657452,-3.906109,0.200481,0.048510,0.013754,-0.415205,0.089457,0.273883,...,-1.876145,-1.285125,-0.262280,-0.726266,-0.179718,-0.181448,3.599520,1.947164,2.446037,-0.007005
2023-11-20 00:00:00+00:00,ZRXUSDT,2.659109,-2.900207,-0.704719,-3.857578,-3.198881,0.147065,-0.106040,-0.386359,-0.024935,0.805087,...,-1.187991,-0.596983,-0.625408,-2.571711,-0.198680,-0.209446,2.944125,3.235569,2.244819,0.010330


In [149]:
def _get_X_y(df):
    dfX = df.drop(columns=[col for col in df.columns if col.startswith('fwdret')])
    feature_names = list(dfX.columns)
    dfX = dfX.unstack()
    X = dfX.values.reshape(len(dfX), len(feature_names), -1)

    y = df[['fwdret1']].unstack().to_numpy()
    return X, y, feature_names

In [150]:
X, y, feature_names = _get_X_y(df)
print(X.shape, y.shape, feature_names)

idx_netbuyquo3 = feature_names.index('netbuyquo3')
y_pred = X[:, idx_netbuyquo3, :]
fit = _quantile35_longshort_fee(y, y_pred, np.ones(X.shape[0]))
print(fit)

(4200, 951, 237) (4200, 237) ['AR_26', 'BCKRET1_ALMA_10_6.0_0.85', 'BCKRET1_ANGLEd_1', 'BCKRET1_APO_12_26', 'BCKRET1_BIAS_SMA_26', 'BCKRET1_CFO_9', 'BCKRET1_CG_10', 'BCKRET1_CMO_14', 'BCKRET1_COPC_11_14_10', 'BCKRET1_CTI_12', 'BCKRET1_DEMA_10', 'BCKRET1_EMA_10', 'BCKRET1_ER_10', 'BCKRET1_FWMA_10', 'BCKRET1_HMA_10', 'BCKRET1_KST_10_15_20_30_10_10_10_15', 'BCKRET1_KSTs_9', 'BCKRET1_MACD_12_26_9', 'BCKRET1_MACDh_12_26_9', 'BCKRET1_MACDs_12_26_9', 'BCKRET1_MOM_1', 'BCKRET1_MOM_3', 'BCKRET1_MOM_5', 'BCKRET1_PPO_12_26_9', 'BCKRET1_PPOh_12_26_9', 'BCKRET1_PPOs_12_26_9', 'BCKRET1_PSL_12', 'BCKRET1_QQE_14_5_4.236', 'BCKRET1_QQE_14_5_4.236_RSIMA', 'BCKRET1_QQEl_14_5_4.236', 'BCKRET1_QQEs_14_5_4.236', 'BCKRET1_ROC_1', 'BCKRET1_ROC_3', 'BCKRET1_ROC_5', 'BCKRET1_RSI_14', 'BCKRET1_RSX_14', 'BCKRET1_SMI_5_20_5', 'BCKRET1_SMIo_5_20_5', 'BCKRET1_SMIs_5_20_5', 'BCKRET1_STC_10_12_26_0.5', 'BCKRET1_STCmacd_10_12_26_0.5', 'BCKRET1_STCstoch_10_12_26_0.5', 'BCKRET1_TRIX_30_9', 'BCKRET1_TRIXs_30_9', 'BCKRET1_

In [151]:
for i in range(0, 10):
    fit = _quantile35_longshort_fee(y, y_pred, np.ones(X.shape[0]))
    print(fit)

1.2754288403710974
1.2754288403710974
1.2754288403710974
0
1.2754288403710974
1.2754288403710974
1.2754288403710974
1.2754288403710974
1.2754288403710974
1.2754288403710974


In [173]:
_annulization = 365 * 3 #默认8h  #TODO
_fee = 0.001

def is_bad_data(y_pred, thresh=0.7):
    total_cs = y_pred.shape[0]
    all_na_cs = np.sum(np.all(np.isnan(y_pred), axis=1))
    return all_na_cs / total_cs >= thresh

def turnover_rate(df):
    df = df.notna()
    per_bar_changes = df.diff().abs().sum(axis=1) / 2
    per_bar_count = df.sum(axis=1)
    rates = per_bar_changes / per_bar_count.shift()
    avg_rate = rates.mean()
    return avg_rate

def compute_quantile_rets(y, y_pred, w, quantiles):
    t1 = perf_counter()
    y_pred = y_pred[w.astype(bool)]
    y = y[w.astype(bool)]
    if np.all(np.isnan(y_pred)):
        return None, None
    
    rets = pd.DataFrame(y)
    factor = pd.DataFrame(y_pred)

    ## use y (return) to mask y_pred to set 0 on all invaid cells to nan
    factor = factor.mask(rets.isna())
    
    groups = np.array(range(quantiles)) + 1
    t2 = perf_counter()
    print(f'qtl1: {t2-t1}')
    
    t1 = perf_counter()
    try:
        factor_quantiles = (
            factor
            .rank(axis=1,method='first')
            .dropna(axis=0, how='all')
            .apply(pd.qcut, q=quantiles, labels=groups, axis=1, duplicates='drop')
            )
    except:
        return None, None
    t2 = perf_counter()
    print(f'qtl2: {t2-t1}')
    
    t1 = perf_counter()
    stacked_rets = rets.stack()
    stacked_factor_quantiles = factor_quantiles.stack()
    grouped_returns = (
        stacked_rets
        .groupby([stacked_rets.index.get_level_values(0), stacked_factor_quantiles])
        .mean()
        .unstack()
        .mean()
        * _annulization
        ) 
    t2 = perf_counter()
    print(f'qtl3: {t2-t1}')
    return grouped_returns, factor_quantiles

def compute_quantile_rets_new(y, y_pred, w, quantiles):
    # t1 = perf_counter()
    y_pred = y_pred[w.astype(bool)]
    y = y[w.astype(bool)]
    if np.all(np.isnan(y_pred)):
        return None, None
    
    rets = pd.DataFrame(y)
    factor = pd.DataFrame(y_pred)

    ## use y (return) to mask y_pred to set 0 on all invaid cells to nan
    factor = factor.mask(rets.isna())
    # t2 = perf_counter()
    # print(f'qtl1: {t2-t1}')
    
    # t1 = perf_counter()
    ret_qtl = []
    for _, grp in factor.stack().dropna().groupby(level=0, group_keys=True):
        ranks = grp.rank()
        bins = np.linspace(ranks.min(), ranks.max(), quantiles+1)
        qtl = np.searchsorted(bins, ranks, side='left')
        qtl[qtl==0] = 1
        ret_qtl.append(pd.Series(qtl, index=ranks.index))

    factor_quantiles = pd.concat(ret_qtl)
    # display(factor_quantiles)
    # t2 = perf_counter()
    # print(f'qtl2: {t2-t1}')
    
    # t1 = perf_counter()
    stacked_rets = rets.stack()
    stacked_factor_quantiles = factor_quantiles
    grouped_returns = (
        stacked_rets
        .groupby([stacked_rets.index.get_level_values(0), stacked_factor_quantiles])
        .mean()
        .unstack()
        .mean()
        * _annulization
        ) 
    # t2 = perf_counter()
    # print(f'qtl3: {t2-t1}')
    return grouped_returns, factor_quantiles.unstack()

def _quantile35_longshort_fee_new(y, y_pred, w):
    if is_bad_data(y_pred):
        return 0
    
    quantiles = 35
    # t1 = perf_counter()
    res, factor_quantiles = compute_quantile_rets_new(y, y_pred, w, quantiles)
    # t2 = perf_counter()
    # print(f'2: {t2-t1}')

    t1 = perf_counter()
    if res is None or factor_quantiles is None:
        return 0
    else:
        if quantiles not in res.index or 1 not in res.index:
            return 0
        ret = abs(res[quantiles] - res[1]) / 2 # annualized longshort (not full time span）
        if np.isnan(ret):
            return 0
        else:
            
            long_turnover = turnover_rate(factor_quantiles[factor_quantiles == quantiles])
            short_turnover = turnover_rate(factor_quantiles[factor_quantiles == 1])
            avg_turnover = (long_turnover + short_turnover) / 2
            ret_fee = ret - avg_turnover * _fee * _annulization
            t2 = perf_counter()
            print(f'3: {t2-t1}')
            return ret_fee

In [174]:
%timeit print(_quantile35_longshort_fee(y, y_pred, np.ones(X.shape[0])))
%timeit print(_quantile35_longshort_fee_new(y, y_pred, np.ones(X.shape[0])))

3: 0.13308924401644617
1.2754288403710974
3: 0.12249807803891599
1.2754288403710974
3: 0.1223179129883647
1.2754288403710974
3: 0.1221340789925307
1.2754288403710974
3: 0.12701961293350905
1.2754288403710974
3: 0.12540584604721516
1.2754288403710974
3: 0.12413793196901679
1.2754288403710974
3: 0.12159713893197477
1.2754288403710974
616 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [142]:
%timeit _, _ = compute_quantile_rets_new(y, y_pred, np.ones(X.shape[0]), 35)
%timeit _, _ = compute_quantile_rets(y, y_pred, np.ones(X.shape[0]), 35)

468 ms ± 6.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
qtl1: 0.004589564981870353
qtl2: 1.2510854239808396
qtl3: 0.046164728002622724
qtl1: 0.00461682491004467
qtl2: 1.2505385249387473
qtl3: 0.04593750403728336
qtl1: 0.0045577839482575655
qtl2: 1.25119592808187
qtl3: 0.047377828042954206
qtl1: 0.004794266074895859
qtl2: 1.2678952139103785
qtl3: 0.045802501030266285
qtl1: 0.004584433976560831
qtl2: 1.2859044750221074
qtl3: 0.0475420180009678
qtl1: 0.004624694935046136
qtl2: 1.2510654639918357
qtl3: 0.04568166995886713
qtl1: 0.004516743938438594
qtl2: 1.2346759809879586
qtl3: 0.04637467593420297
qtl1: 0.004527564044110477
qtl2: 1.2441464299336076
qtl3: 0.045325334998779
1.31 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
